#  RAG 체인 구성

---

# 환경 설정 및 준비

`(1) Env 환경변수`

In [38]:
from dotenv import load_dotenv
load_dotenv()

True

`(2) 기본 라이브러리`

In [39]:
import os
from glob import glob

from pprint import pprint
import json

# 다양한 문서 형식 처리하기

- 역할: Document Loader는 다양한 소스에서 문서를 로드
- 구현: 
    - Document Loader는 BaseLoader 인터페이스를 통해서 구현
    - `.load()` 또는 `.lazy_load()` 메서드를 통해 동일한 방식으로 호출 
    - 대용량 데이터셋의 경우 메모리 효율을 위해 `.lazy_load()`를 사용하는 것을 권장 
- 종류:
    - PDF 파일 로더
    - 웹 페이지 로더 
    - CSV 데이터 로더
    - 디렉토리 로더
    - HTML 데이터 로더
    - JSON 데이터 로더
    - Markdown 데이터 로더
    - Microsoft Office 데이터 로더

### 1. **웹 문서 로더**

- uv add bs4 langchain_community

In [40]:
from langchain_community.document_loaders import WebBaseLoader

# 기본적인 텍스트 추출
web_loader = WebBaseLoader(
    web_paths=[
        "https://python.langchain.com/", 
        "https://js.langchain.com/",
        ],
    )

# 동기 로딩
web_docs = web_loader.load()

len(web_docs)

2

In [41]:
web_docs[0].metadata

{'source': 'https://python.langchain.com/',
 'title': 'LangChain overview - Docs by LangChain',
 'language': 'en'}

In [42]:
print(web_docs[0].page_content)

LangChain overview - Docs by LangChainSkip to main content🚀 Share how you're building agents for a chance to win LangChain swag!Docs by LangChain home pageLangChain + LangGraphSearch...⌘KAsk AIGitHubTry LangSmithTry LangSmithSearch...NavigationLangChain overviewLangChainLangGraphDeep AgentsIntegrationsLearnReferenceContributePythonOverviewGet startedInstallQuickstartPhilosophyCore componentsAgentsModelsMessagesToolsShort-term memoryStreamingStructured outputMiddlewareOverviewBuilt-in middlewareCustom middlewareAdvanced usageGuardrailsRuntimeContext engineeringModel Context Protocol (MCP)Human-in-the-loopMulti-agentRetrievalLong-term memoryAgent developmentLangSmith StudioTestAgent Chat UIDeploy with LangSmithDeploymentObservabilityOn this page Create an agent Core benefitsLangChain overviewCopy pageCopy pageLangChain v1.x is now available!For a complete list of changes and instructions on how to upgrade your code, see the release notes and migration guide.If you encounter any issues or

### 2. **CSV 파일 로더**

In [43]:
from langchain_community.document_loaders.csv_loader import CSVLoader

# 기본 파일 로드
csv_loader = CSVLoader("./data/kbo_teams_2023.csv")
csv_docs = csv_loader.load()

print("문서의 수:", len(csv_docs))
print("-" * 50)
print("처음 문서의 메타데이터: \n", csv_docs[0].metadata)
print("-" * 50)
print("처음 문서의 내용: \n", csv_docs[0].page_content)

문서의 수: 10
--------------------------------------------------
처음 문서의 메타데이터: 
 {'source': './data/kbo_teams_2023.csv', 'row': 0}
--------------------------------------------------
처음 문서의 내용: 
 Team: KIA 타이거즈
City: 광주
Founded: 1982
Home Stadium: 광주-기아 챔피언스 필드
Championships: 11
Introduction: KBO 리그의 전통 강호로, 역대 최다 우승 기록을 보유하고 있다. '타이거즈 스피릿'으로 유명하며, 양현종, 안치홍 등 스타 선수들을 배출했다. 광주를 연고로 하는 유일한 프로야구팀으로 지역 사랑이 강하다.


### 3. **PDF 파일 로더**

- uv add langchain_community pypdf

In [44]:
from langchain_community.document_loaders import PyPDFLoader

# PDF 로더 초기화 (Tesla Annual Report 문서)
pdf_loader = PyPDFLoader('./data/tsla-20241231-gen.pdf')
         
# 동기 로딩
pdf_docs = pdf_loader.load()
print(f'PDF 문서 개수: {len(pdf_docs)}')

PDF 문서 개수: 144


# 텍스트 분할(Text Splitting) 

- 대규모 텍스트 문서를 처리할 때 매우 중요한 전처리 단계
- 고려사항:
    1. 문서의 구조와 형식
    2. 원하는 청크 크기
    3. 문맥 보존의 중요도
    4. 처리 속도 

In [45]:
print(f'첫 번째 문서: {pdf_docs[0]}')

첫 번째 문서: page_content='UNITED	STATES
SECURITIES	AND	EXCHANGE	COMMISSION
Washington,	D.C.	20549
FORM	
10-K
(Mark	One)
x
ANNUAL	REPORT	PURSUANT	TO	SECTION	13	OR	15(d)	OF	THE	SECURITIES	EXCHANGE	ACT	OF	1934
For	the	fiscal	year	ended	
December	31
,	2024
OR
o
TRANSITION	REPORT	PURSUANT	TO	SECTION	13	OR	15(d)	OF	THE	SECURITIES	EXCHANGE	ACT	OF	1934
For	the	transition	period	from	_________	to	_________
Commission	File	Number:	
001-34756
Tesla,	Inc.
(Exact	name	of	registrant	as	specified	in	its	charter)
Texas
91-2197729
(State	or	other	jurisdiction	of
incorporation	or	organization)
(I.R.S.	Employer
Identification	No.)
1	Tesla	Road
Austin
,	
Texas
78725
(Address	of	principal	executive	offices)
(Zip	Code)
(
512
)	
516-8177
(Registrant’s	telephone	number,	including	area	code)
Securities	registered	pursuant	to	Section	12(b)	of	the	Act:
Title	of	each	class
Trading	Symbol(s)
Name	of	each	exchange	on	which	registered
Common	stock
TSLA
The	Nasdaq	Global	Select	Market
Securities	registered	pursuant	to	S

In [46]:
long_text = pdf_docs[0].page_content
print(f'첫 번째 문서의 텍스트 길이: {len(long_text)}')

첫 번째 문서의 텍스트 길이: 2450


### 1. **CharacterTextSplitter**
- 가장 기본적인 분할 방식
- 문자 수를 기준으로 텍스트를 분할
- 단순하지만 문맥을 고려하지 않는다는 단점이 있음

- 설치: pip install langchain_text_splitters 또는 uv add langchain_text_splitters

In [47]:
from langchain_text_splitters import CharacterTextSplitter 

# 텍스트 분할기 초기화 (기본 설정값 적용 )
text_splitter = CharacterTextSplitter(

    # CharacterTextSplitter의 기본 설정값
    separator = "\n\n",         # 청크 구분자: 두 개의 개행문자
    is_separator_regex = False,  # 구분자가 정규식인지 여부

    # TextSplitter의 기본 설정값
    chunk_size = 500,          # 청크 길이
    chunk_overlap = 100,        # 청크 중첩
    length_function = len,      # 길이 함수 (문자열 길이)
)

# 텍스트 분할 - split_text() 메서드 사용
texts = text_splitter.split_text(long_text)

# 분할된 텍스트 개수 출력
print(f'분할된 텍스트 개수: {len(texts)}')

# 첫 번째 분할된 텍스트 출력
print(f'첫 번째 분할된 텍스트: {texts[0]}')

분할된 텍스트 개수: 1
첫 번째 분할된 텍스트: UNITED	STATES
SECURITIES	AND	EXCHANGE	COMMISSION
Washington,	D.C.	20549
FORM	
10-K
(Mark	One)
x
ANNUAL	REPORT	PURSUANT	TO	SECTION	13	OR	15(d)	OF	THE	SECURITIES	EXCHANGE	ACT	OF	1934
For	the	fiscal	year	ended	
December	31
,	2024
OR
o
TRANSITION	REPORT	PURSUANT	TO	SECTION	13	OR	15(d)	OF	THE	SECURITIES	EXCHANGE	ACT	OF	1934
For	the	transition	period	from	_________	to	_________
Commission	File	Number:	
001-34756
Tesla,	Inc.
(Exact	name	of	registrant	as	specified	in	its	charter)
Texas
91-2197729
(State	or	other	jurisdiction	of
incorporation	or	organization)
(I.R.S.	Employer
Identification	No.)
1	Tesla	Road
Austin
,	
Texas
78725
(Address	of	principal	executive	offices)
(Zip	Code)
(
512
)	
516-8177
(Registrant’s	telephone	number,	including	area	code)
Securities	registered	pursuant	to	Section	12(b)	of	the	Act:
Title	of	each	class
Trading	Symbol(s)
Name	of	each	exchange	on	which	registered
Common	stock
TSLA
The	Nasdaq	Global	Select	Market
Securities	registered	pursuant

In [48]:
from langchain_text_splitters import CharacterTextSplitter

# 문장 구분자를 개행문자로 설정
text_splitter = CharacterTextSplitter(
    separator = "\n",        # 청크 구분자: 개행문자
    chunk_size = 500,       # 청크 길이
    chunk_overlap = 100      # 청크 중첩
)

# split_documents() 메서드 사용 : Document 객체를 여러 개의 작은 청크 문서로 분할
chunks = text_splitter.split_documents([pdf_docs[0]])   # 첫 번째 문서만 분할

# 분할된 텍스트 개수 출력
print(f'분할된 텍스트 개수: {len(chunks)}')

# 각 청크의 텍스트 길이 출력
for i, chunk in enumerate(chunks):
    print(f'청크 {i+1}의 텍스트 길이: {len(chunk.page_content)}')

# 첫 번째 청크의 텍스트 출력
print(f'첫 번째 청크의 텍스트: {chunks[0].page_content}')

분할된 텍스트 개수: 6
청크 1의 텍스트 길이: 493
청크 2의 텍스트 길이: 491
청크 3의 텍스트 길이: 464
청크 4의 텍스트 길이: 384
청크 5의 텍스트 길이: 342
청크 6의 텍스트 길이: 487
첫 번째 청크의 텍스트: UNITED	STATES
SECURITIES	AND	EXCHANGE	COMMISSION
Washington,	D.C.	20549
FORM	
10-K
(Mark	One)
x
ANNUAL	REPORT	PURSUANT	TO	SECTION	13	OR	15(d)	OF	THE	SECURITIES	EXCHANGE	ACT	OF	1934
For	the	fiscal	year	ended	
December	31
,	2024
OR
o
TRANSITION	REPORT	PURSUANT	TO	SECTION	13	OR	15(d)	OF	THE	SECURITIES	EXCHANGE	ACT	OF	1934
For	the	transition	period	from	_________	to	_________
Commission	File	Number:	
001-34756
Tesla,	Inc.
(Exact	name	of	registrant	as	specified	in	its	charter)
Texas
91-2197729


### 2. **RecursiveCharacterTextSplitter**

- 재귀적으로 텍스트를 분할
- 구분자를 순차적으로 적용하여 큰 청크에서 시작하여 점진적으로 더 작은 단위로 분할
- 문맥을 더 잘 보존할 수 있음  


In [49]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 재귀적 텍스트 분할기 초기화
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,             # 청크 크기
    chunk_overlap=100,           # 청크 중 중복되는 부분 크기
    length_function=len,         # 글자 수를 기준으로 분할
    separators=["\n\n", "\n", " ", ""],  # 구분자 - 재귀적으로 순차적으로 적용 
)

# split_documents() 메서드 사용 : Document 객체를 여러 개의 작은 청크 문서로 분할
chunks = text_splitter.split_documents(pdf_docs)
print(f"생성된 텍스트 청크 수: {len(chunks)}")
print(f"각 청크의 길이: {list(len(chunk.page_content) for chunk in chunks)}")
print()

# 각 청크의 시작 부분과 끝 부분 확인 - 5개 청크만 출력
for chunk in chunks[:5]:
    print(chunk.page_content[:200])
    print("-" * 100)
    print(chunk.page_content[-200:])
    print("=" * 100)
    print()

생성된 텍스트 청크 수: 1190
각 청크의 길이: [493, 491, 464, 384, 342, 487, 428, 407, 461, 363, 383, 442, 460, 489, 464, 167, 489, 457, 451, 447, 302, 496, 441, 361, 393, 400, 488, 379, 371, 486, 452, 102, 482, 490, 386, 495, 458, 468, 445, 433, 341, 357, 395, 492, 449, 478, 411, 493, 392, 366, 402, 498, 443, 458, 364, 487, 357, 400, 394, 443, 446, 451, 480, 426, 486, 397, 485, 430, 119, 461, 491, 485, 353, 464, 482, 465, 441, 449, 347, 476, 394, 387, 440, 440, 442, 492, 359, 451, 387, 420, 369, 430, 452, 428, 460, 446, 491, 382, 455, 437, 357, 456, 350, 377, 450, 443, 435, 454, 412, 415, 449, 482, 462, 490, 383, 420, 436, 420, 471, 423, 489, 134, 419, 437, 408, 433, 429, 494, 434, 456, 452, 447, 62, 467, 381, 485, 436, 413, 475, 472, 419, 401, 491, 456, 85, 472, 442, 373, 429, 457, 351, 466, 450, 447, 457, 418, 427, 417, 451, 470, 459, 424, 378, 485, 468, 374, 361, 486, 458, 427, 471, 458, 338, 393, 455, 477, 380, 451, 488, 480, 441, 436, 453, 444, 446, 59, 484, 376, 456, 352, 402, 346, 458, 446, 371

# 문서 임베딩(Document Embedding)

- 개념: 
    - 텍스트를 벡터(숫자 배열)로 변환하는 과정
    - 문서의 의미적 특성을 수치화하여 컴퓨터가 이해하고 처리할 수 있는 형태로 변환 

- 목적:
    - 텍스트 간 유사도 계산 가능
    - 벡터 데이터베이스 저장 및 검색
    - 의미 기반 문서 검색 구현

### 1. **OpenAI**

- LangChain에서 가장 널리 사용되는 임베딩 모델 중 하나

- 주요 특징:
    1. 고품질의 임베딩 생성
    2. 다양한 언어 지원 (다국어 지원)
    3. 일관된 성능
    4. 손쉬운 통합

- 사용시 주의사항:
    1. API 키 설정이 필요 (환경 변수 OPENAI_API_KEY)
    2. API 사용량에 따른 비용 발생
    3. 긴 텍스트는 자동으로 분할되지 않으므로 필요시 TextSplitter를 사용


- 모델별 특징
    ```
    모델                    페이지/달러    MTEB 성능     최대입력
    text-embedding-3-small  62,500       62.3%       8191
    text-embedding-3-large   9,615       64.6%       8191
    text-embedding-ada-002  12,500       61.0%       8191
    ```

- 임베딩 벡터 특성
    1. small: 1536 차원
    1. large: 3072 차원
    1. dimensions 파라미터로 차원 축소 가능

`(1) embedding 모델`

In [50]:
from langchain_openai import OpenAIEmbeddings

# OpenAIEmbeddings 모델 생성
embeddings_model = OpenAIEmbeddings(
    model="text-embedding-3-large",  # 사용할 모델 이름
    dimensions=None, # 원하는 임베딩 차원 수를 지정 가능 (기본값: None)
    )

# 임베딩 객체 출력
embeddings_model

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x11a463680>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x11a433fe0>, model='text-embedding-3-large', dimensions=None, deployment='text-embedding-ada-002', openai_api_version=None, openai_api_base=None, openai_api_type=None, openai_proxy=None, embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

In [51]:
# 임베딩 모델의 컨텍스트 길이 확인
embeddings_model.embedding_ctx_length

8191

In [52]:
from langchain_openai import OpenAIEmbeddings

# OpenAIEmbeddings 모델 생성할 때 임베딩 차원을 지정하는 예시
embeddings_openai = OpenAIEmbeddings(
    model="text-embedding-3-small",  # 사용할 모델 이름
    dimensions=1536, # 원하는 임베딩 차원 수를 지정 가능 (기본값: None)
    )

# 임베딩 모델의 임베딩 차원 확인 
embeddings_openai.dimensions

1536

`(2) embed_documents 사용`

In [53]:
# 문們서 컬렉션
documents = [
    "인人공지능은 컴퓨터 과학의 한 분야입니다.",
    "머신러닝은 인공지능의 하위 분야입니다.",
    "딥러닝은 머신러닝의 한 종류입니다.",
    "자연어 처리는 컴퓨터가 인간의 언어를 이해하고 생성하는 기술입니다.",
    "컴퓨터 비전은 컴퓨터가 디지털 이미지나 비디오를 이해하는 방법을 연구합니다."
]

# 문서 임베딩
document_embeddings_openai = embeddings_openai.embed_documents(documents)

# 임베딩 결과 출력
print(f"임베딩 벡터의 개수: {len(document_embeddings_openai)}")
print(f"임베딩 벡터의 차원: {len(document_embeddings_openai[0])}")
print(document_embeddings_openai[0])

임베딩 벡터의 개수: 5
임베딩 벡터의 차원: 1536
[0.019355662167072296, 0.010378574021160603, 0.026495667174458504, 0.04916565492749214, 0.015492105856537819, -0.0340333916246891, -0.006908001843839884, 0.025472961366176605, -0.027972908690571785, -0.048597484827041626, 0.01948823593556881, -0.00604627700522542, 0.006278280168771744, -0.024469193071126938, 0.0017565926536917686, -0.011959981173276901, -0.05560491606593132, -0.0037925357464700937, 0.05272618681192398, -0.06302900612354279, 0.006600243039429188, -0.011941042728722095, -0.01248080376535654, -0.04075673595070839, 0.008247936144471169, -0.04587026685476303, 0.04219610244035721, 0.026552483439445496, -0.016789427027106285, -0.036646973341703415, 0.010189183987677097, -0.021457891911268234, -0.025397205725312233, -0.043559711426496506, 0.02564341202378273, 0.028275933116674423, -0.01445046067237854, 0.015302715823054314, 0.008257405832409859, 0.0010759722208604217, 0.0061172982677817345, -0.025548717007040977, -0.0015222225338220596, 0.0356432

`(3) embed_query 사용`

In [54]:
embedded_query_openai = embeddings_openai.embed_query("인공지능이란 무엇인가요?")

# 쿼리 임베딩 결과 출력
print(f"쿼리 임베딩 벡터의 차원: {len(embedded_query_openai)}")
print(embedded_query_openai)

쿼리 임베딩 벡터의 차원: 1536
[-0.02255643531680107, 0.022176697850227356, 0.0004912859876640141, 0.005667589604854584, 0.012189589440822601, -0.04480908066034317, -0.026277868077158928, 0.035809293389320374, -0.002643925603479147, 0.014800287783145905, -0.002100425772368908, 0.0008621239103376865, -0.004865393042564392, -0.07359322160482407, 0.009018776938319206, -0.01542685553431511, -0.05980873107910156, -0.022442514076828957, 0.022423528134822845, -0.0692262351512909, -0.029220838099718094, 0.023239964619278908, -0.036853570491075516, 0.00411778362467885, 0.011145309545099735, -0.0526316836476326, 0.010727598331868649, -3.0946419428801164e-05, -0.010803545825183392, -0.03410046920180321, 0.02529054880142212, -0.018910951912403107, -0.0014999649720266461, -0.05897330865263939, 0.04989757016301155, -0.0034152683801949024, -0.0027602205518633127, -0.0083684753626585, -0.004597202874720097, 0.02280326560139656, -0.011885798536241055, 0.0379737988114357, 0.0032989736646413803, 0.03520170971751213

`(4) 유사도 기반 검색`

In [55]:
from langchain_community.utils.math import cosine_similarity
import numpy as np

# 쿼리와 가장 유사한 문서 찾기 함수
def find_most_similar(
        query: str, 
        doc_embeddings: np.ndarray,
        embeddings_model=OpenAIEmbeddings(model="text-embedding-3-small")
        ) -> tuple[str, float]:
    
    # 쿼리 임베딩: OpenAI 임베딩 사용 
    query_embedding = embeddings_model.embed_query(query)

    # 코사인 유사도 계산
    similarities = cosine_similarity([query_embedding], doc_embeddings)[0]

    # 가장 유사한 문서 인덱스 찾기
    most_similar_idx = np.argmax(similarities)

    # 가장 유사한 문서와 유사도 반환: 문서, 유사도
    return documents[most_similar_idx], similarities[most_similar_idx]

# 예제 쿼리
queries = [
    "인공지능이란 무엇인가요?",
    "딥러닝과 머신러닝의 관계는 어떻게 되나요?",
    "컴퓨터가 이미지를 이해하는 방법은?"
]

# 각 쿼리에 대해 가장 유사한 문서 찾기
for query in queries:
    most_similar_doc, similarity = find_most_similar(
        query, 
        document_embeddings_openai,  
        embeddings_model=embeddings_openai
        )
    print(f"쿼리: {query}")
    print(f"가장 유사한 문서: {most_similar_doc}")
    print(f"유사도: {similarity:.4f}")
    print()

쿼리: 인공지능이란 무엇인가요?
가장 유사한 문서: 인人공지능은 컴퓨터 과학의 한 분야입니다.
유사도: 0.4885

쿼리: 딥러닝과 머신러닝의 관계는 어떻게 되나요?
가장 유사한 문서: 딥러닝은 머신러닝의 한 종류입니다.
유사도: 0.6815

쿼리: 컴퓨터가 이미지를 이해하는 방법은?
가장 유사한 문서: 컴퓨터 비전은 컴퓨터가 디지털 이미지나 비디오를 이해하는 방법을 연구합니다.
유사도: 0.7051



### 2. **Huggingface**

- LangChain에서 오픈소스 기반의 대표적인 임베딩 모델

- 주요 특징:
    1. 모델별로 다양한 차원 제공 (128 ~ 1024)
    2. sentence-transformers 기반 구현
    3. BERT 계열 모델 구조 사용
    4. 코사인 유사도 기반 검색 최적화
    5. 커스텀 모델 학습 및 적용 가능
    6. 무료 사용 가능 (API 비용 없음)

- 사용시 주의사항:
    1. 로컬 컴퓨팅 자원 필요 (CPU/GPU)
    2. 초기 모델 다운로드 시간 소요
    3. 메모리 사용량 고려 필요
    4. transformers 라이브러리 설치 필요

- 대표적인 임베딩 모델:
    ```
    모델                            차원      언어             특징      
    all-MiniLM-L6-v2               384     다국어     빠른 속도, 적은 메모리
    all-mpnet-base-v2              768     다국어     높은 성능, 중간 크기
    multilingual-e5-large         1024     다국어     최고 성능, 큰 메모리
    ```

`(1) embedding 모델`

- langchain_huggingface 설치 필요

In [57]:
from langchain_huggingface import HuggingFaceEmbeddings  

# Hugging Face의 임베딩 모델 생성
embeddings_bgem3 = HuggingFaceEmbeddings(
    model_name="BAAI/bge-m3",          # 사용할 모델 이름 - BAAI BGE-m3 모델 (한국어 성능 우수)
    # model_kwargs={'device': 'cuda'}  # GPU 사용시
    # model_kwargs={'device': 'mps'}   # Apple Silicon 사용시
)

# 임베딩 객체 출력
embeddings_bgem3

/Users/jonghwanbyun/edu/fin-llm/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


HuggingFaceEmbeddings(model_name='BAAI/bge-m3', cache_folder=None, model_kwargs={}, encode_kwargs={}, query_encode_kwargs={}, multi_process=False, show_progress=False)

`(2) embed_documents 사용`

In [58]:
# 문서 임베딩
document_embeddings_bgem3 = embeddings_bgem3.embed_documents(documents)

# 임베딩 결과 출력
print(f"임베딩 벡터의 개수: {len(document_embeddings_bgem3)}")
print(f"임베딩 벡터의 차원: {len(document_embeddings_bgem3[0])}")
print(document_embeddings_bgem3[0])

임베딩 벡터의 개수: 5
임베딩 벡터의 차원: 1024
[-0.05196467414498329, -5.574165334110148e-05, -0.0010357709834352136, -0.018594982102513313, -0.018673693761229515, -0.01022404432296753, 0.006904600653797388, -0.006033142097294331, 0.043813832104206085, -0.009522098116576672, -0.039305392652750015, -0.024427542462944984, -0.0031465501524508, 0.03612295165657997, 0.010898453183472157, 0.023502983152866364, 0.0219043530523777, -0.037266023457050323, -0.01213859859853983, -0.024659190326929092, 0.022374093532562256, 0.009629892185330391, 0.010453864000737667, 0.021524028852581978, 0.00826297514140606, -0.023691346868872643, -0.017083479091525078, -0.005562406498938799, 0.01699906773865223, -0.014509140513837337, 0.012067070230841637, -0.04188072308897972, 0.027258148416876793, -0.055022843182086945, -0.08650309592485428, -0.028797436505556107, -0.005660074297338724, -0.02426738291978836, -0.04322672262787819, 0.007353850174695253, -0.03529975190758705, -0.03695284575223923, 0.0022761456202715635, -0.04200

`(3) embed_query 사용`

In [ ]:
embedded_query = embeddings_bgem3.embed_query("인공지능이란 무엇인가요?")

# 쿼리 임베딩 결과 출력
print(f"쿼리 임베딩 벡터의 차원: {len(embedded_query)}")
print(embedded_query)

`(4) 유사도 기반 검색`

In [ ]:
# 예제 쿼리
queries = [
    "인공지능이란 무엇인가요?",
    "딥러닝과 머신러닝의 관계는 어떻게 되나요?",
    "컴퓨터가 이미지를 이해하는 방법은?"
]

# 각 쿼리에 대해 가장 유사한 문서 찾기
for query in queries:
    most_similar_doc, similarity = find_most_similar(query, document_embeddings_bgem3, embeddings_model=embeddings_bgem3) 
    print(f"쿼리: {query}")
    print(f"가장 유사한 문서: {most_similar_doc}")
    print(f"유사도: {similarity:.4f}")
    print()

# 벡터 저장소 (Vector Store)

- 개념:
    - 벡터화된 데이터를 효율적으로 저장하고 검색하기 위한 특수 데이터베이스 시스템
    - 텍스트나 이미지 등의 비정형 데이터를 고차원 벡터 공간에 매핑하여 저장
    - 유사도 기반 검색을 통해 의미적으로 가까운 데이터를 빠르게 검색 가능 

- LangChain의 벡터 저장소 종류:
    - **Chroma**: 경량화된 임베딩 데이터베이스로 로컬 개발에 적합
    - **FAISS**: Facebook AI가 개발한 고성능 유사도 검색 라이브러리
    - **Pinecone**: 완전 관리형 벡터 데이터베이스 서비스
    - Milvus: 분산 벡터 데이터베이스로 대규모 데이터 처리에 적합
    - PostgreSQL: pgvector 확장을 통해 벡터 저장 및 검색 기능을 제공

- 주요 기능:
    - 벡터 색인화: 효율적인 검색을 위한 데이터 구조화를 수행
    - 근접 이웃 검색: 주어진 쿼리와 가장 유사한 벡터들을 검색 
    - 메타데이터 관리: 벡터와 관련된 부가 정보를 함께 저장하고 검색

- 사용 사례:
    - 시맨틱 문서 검색: 문서의 의미를 이해하여 검색
    - 추천 시스템: 유사한 아이템을 추천
    - 중복 데이터 감지: 유사한 콘텐츠를 검색 
    - 질의응답 시스템: 관련 문서에서 답변을 생성하는데 필요한 근거를 검색 

### **Chroma**

- 사용자 편의성이 우수한 오픈소스 벡터 저장소
- `langchain-chroma` 패키지 설치

`(1) 벡터 저장소 초기화`

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

# PDF 로더 초기화
pdf_loader = PyPDFLoader('./data/tsla-20241231-gen.pdf')

# 동기 로딩
pdf_docs = pdf_loader.load()
print(f'PDF 문서 개수: {len(pdf_docs)}')

len(pdf_docs)

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# TikToken 인코더를 사용하여 재귀적 텍스트 분할기 초기화 (토큰 수 기준 분할)
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    encoding_name="cl100k_base", 
    chunk_size=1000,  # 청크 크기: 1000 토큰
    chunk_overlap=200,  # 청크 중첩: 200 토큰
)

# split_documents() 메서드 사용 : Document 객체를 여러 개의 작은 청크 문서로 분할
chunks = text_splitter.split_documents(pdf_docs) 

print(f"생성된 청크 수: {len(chunks)}")
print(f"각 청크의 길이: {list(len(chunk.page_content) for chunk in chunks)}")

# 각 청크의 시작 부분과 끝 부분 확인
for chunk in chunks[:5]:
    print(f"{chunk.page_content[:100]}...{chunk.page_content[-100:]}")
    print("="*100)

In [ ]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

# OpenAI 임베딩 모델 생성
embeddings_openai = OpenAIEmbeddings(
    model="text-embedding-3-small",  # 사용할 모델 이름)
)

# Chroma 벡터 저장소 생성하기
chroma_db = Chroma.from_documents(  
    documents=chunks,
    embedding=embeddings_openai,    # OpenAI 임베딩 사용
    collection_name="tsla-20241231-gen",    # 컬렉션 이름
    persist_directory="./chroma_db",
    collection_metadata = {'hnsw:space': 'cosine'}, # l2, ip, cosine 중에서 선택 
)

In [ ]:
# 문서 개수 확인
chroma_db._collection.count()

`(2) 벡터 저장소 로드`  

In [ ]:
from langchain_chroma import Chroma

# 저장된 벡터 저장소를 가져오기
chroma_db = Chroma(
    collection_name="tsla-20241231-gen",  # 컬렉션 이름
    embedding_function=embeddings_openai,
    persist_directory="./chroma_db",)

In [ ]:
# 문서 개수 확인
chroma_db._collection.count()

`(3) 문서 검색`  

- 유사도 검색: 주어진 쿼리와 가장 유사한 문서를 반환
-  k=5는 상위 5개의 결과를 반환하도록 지정
- filter를 사용하여 특정 출처의 문서만 검색 가능

In [ ]:
query = "Tesla의 2024년 매출액과 영업이익은 얼마인가요?"
results = chroma_db.similarity_search(
    query,
    k=5,  # 검색할 문서의 개수
)

print("유사도 검색 결과:")
for doc in results:
    print(f"- {doc.page_content} [출처: {doc.metadata['source']}]")
    print("=" * 100)

# 벡터 저장소 기반 RAG 검색기 (Retriever)



In [ ]:
retriever = chroma_db.as_retriever(
    search_kwargs={"k": 2},
)

query = "Tesla의 2024년 매출액과 영업이익은 얼마인가요?"
retrieved_docs = retriever.invoke(query)

print(f"쿼리: {query}")
print("검색 결과:")
for i, doc in enumerate(retrieved_docs, 1):
    print(f"-{i}-\n{doc.page_content}\n[출처: {doc.metadata['source']}]")
    print("-" * 100)

`(2) MMR(Maximal Marginal Relevance) 검색`

In [ ]:
# MMR - 다양성 고려 (lambda_mult 작을수록 더 다양하게 추출)
retriever = chroma_db.as_retriever(
    search_type='mmr',
    search_kwargs={
        'k': 3,                 # 검색할 문서의 수
        'fetch_k': 8,           # mmr 알고리즘에 전달할 문서의 수 (fetch_k > k)
        'lambda_mult': 0.5,     # 다양성을 고려하는 정도 (1은 최소 다양성, 0은 최대 다양성을 의미. 기본값은 0.5)
        },
)


query = "Tesla의 2024년 매출액과 영업이익은 얼마인가요?"

# 쿼리와 유사한 문서 검색
retrieved_docs = retriever.invoke(query)

print(f"쿼리: {query}")
print("검색 결과:")
for i, doc in enumerate(retrieved_docs, 1):
    score = cosine_similarity(
        [embeddings_openai.embed_query(query)], 
        [embeddings_openai.embed_query(doc.page_content)]
        )[0][0]
    print(f"-{i}-\n{doc.page_content}\n[유사도: {score}]")
    print("-" * 100)

# Naive RAG 구현 

`(1) 벡터 저장소 로드`
- `Chroma` 벡터 저장소를 로드

In [ ]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

# OpenAI 임베딩 모델 생성
embeddings_openai = OpenAIEmbeddings(
    model="text-embedding-3-small",  # 사용할 모델 이름
    )

# 저장된 벡터 저장소를 가져오기
chroma_db = Chroma(
    collection_name="tsla-20241231-gen",  # 컬렉션 이름
    embedding_function=embeddings_openai,
    persist_directory="./chroma_db",
)

# 문서 개수 확인
print(f"저장된 문서 수: {chroma_db._collection.count()}")

`(2) 검색기(Retriever) 초기화`

- mmr 검색을 사용하는 Retriever 사용
- 다양성을 높이는 설정을 사용

In [ ]:
# mmr 검색기 생성
retriever = chroma_db.as_retriever(
    search_type='mmr',
    search_kwargs={
        'k': 7,                  # 검색할 문서의 수
        'fetch_k': 20,           # mmr 알고리즘에 전달할 문서의 수 (fetch_k > k)
        'lambda_mult': 0.9,      # 다양성을 고려하는 정도 (1은 최소 다양성, 0은 최대 다양성을 의미. 기본값은 0.5)
        },
)

In [ ]:
# 검색 테스트 
query = "Tesla의 2024년 매출액과 영업이익은 얼마인가요?"

# 쿼리와 유사한 문서 검색
retrieved_docs = retriever.invoke(query)

print(f"쿼리: {query}")
print("검색 결과:")
for i, doc in enumerate(retrieved_docs, 1):
    print(f"-{i}-\n{doc.page_content}\n[출처: {doc.metadata['source']}]")
    print("-" * 100)

`(3) RAG 프롬프트 구성`

- 작성 기준: 
    - LangChain의 ChatPromptTemplate 클래스 사용
    - 변수 처리는 {context}, {question} 형식 사용
    - 답변은 한글로 출력되도록 프롬프트 작성
    
- 아래 템플릿 코드를 기반으로 다음 내용을 참고하여 작성합니다. 

    1. 프롬프트 구성요소:
        - 작업 지침
        - 컨텍스트 영역
        - 질문 영역
        - 답변 형식 가이드

    2. 작업 지침:
        - 컨텍스트 기반 답변 원칙
        - 외부 지식 사용 제한
        - 불확실성 처리 방법
        - 답변 불가능한 경우의 처리 방법

    3. 답변 형식:
        - 핵심 답변 섹션
        - 근거 제시 섹션
        - 추가 설명 섹션 (필요시)

    4. 제약사항 반영:
        - 답변은 사실에 기반해야 함
        - 추측이나 가정을 최소화해야 함
        - 명확한 근거 제시가 필요함
        - 구조화된 형태로 작성되어야 함

In [ ]:
# Prompt 템플릿 (예시)
from langchain_core.prompts import ChatPromptTemplate

system_prompt = """You are a helpful assistant. 

<Instruction>
1. Answer the question based only on the following context.
2. If the context does not contain the answer, say "적절한 답변을 찾을 수 없습니다."
3. Do not include any additional information.
4. Answer in 한국어.
</Instruction>

"""

user_prompt = """
<Context>
{context}
</Context>

<Question> 
{question}
</Question>
"""

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("user", user_prompt)
])

# 템플릿 출력
prompt.pretty_print()

`(4) RAG 체인 구성`
- LangChain의 LCEL 문법을 사용
- 검색 결과를 프롬프트의 'context'로 전달하고,
- 사용자가 입력한 질문을 그래도 프롬프트의 'question'에 전달
- LLM 설정:
    - ChatOpenAI 사용 ('gpt-4.1-mini' 모델)
    - temperature: 답변의 일관성을 가져가는 설정값을 사용 
    - 기타 필요한 설정 
- 출력 파서: 문자열 부분만 출력되도록 구성 

In [ ]:
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

# LLM 설정
llm = ChatOpenAI(
    model="gpt-4.1-mini",
    temperature=0.7,
    top_p=0.9,
)

# 문서 포맷팅
def format_docs(docs):
    return "\n\n".join([f"{doc.page_content}" for doc in docs])

# RAG 체인 생성
rag_chain = (
    RunnableParallel(
        {
            "context": retriever | format_docs, 
            "question": RunnablePassthrough()
        }
    )
    | prompt
    | llm
    | StrOutputParser()
)

# 체인 실행
query = "Tesla의 2024년 매출액과 영업이익은 얼마인가요?"
output = rag_chain.invoke(query)

print(f"쿼리: {query}")
print("답변:")
print(output)

# [실습] RAG 체인 구성

- RAG 체인을 구성하여 질문에 대한 답변을 생성해보세요.
- 문서 로딩, 임베딩, 벡터 저장소 구축, 검색기 설정 등을 포함하여 전체 RAG 파이프라인을 구현합니다.
- 대상 문서: data/housing_leasing_law.pdf

In [ ]:
# 여기에 코드를 작성하세요